# First Year Project - Project 1 - Corona and Weather

----
**Group 6 (F)**: Bjørn Søvad (bjso@itu.dk), Katarina Kraljevic (katkr@itu.dk), Mirka Katuscáková (katu@itu.dk), Emma Cecilie Bjerring Jensen (emcj@itu.dk), Viggo Yann Unmack Gascou (viga@itu.dk)

----

### Instructions for running this Jupyter Notebook
Make sure to run this notebook in trusted mode. \
This will ensure that the code blocks run correctly and display the output correctly

This is the expected folder structure. In order for this notebook to run correctly this folder structure must be used

```
fyp2022p01g06/
├─ Notebooks/
│  ├─ fyp2022p01g06_Report.ipynb (current location)
├─ Data/
│  ├─ raw/
│  │  ├─ corona/
│  │  ├─ weather/
│  │  ├─ shapefiles/
│  │  ├─ metadata/
│  ├─ external/
│  │  ├─ de_our_world_in_data_covid.csv
│  │  ├─ de_subnational_HDI.csv
│  ├─ processed/
│  │  ├─ ...
│  │  interim/
```

### Required Libraries
If needed you can read more documentation about the different libraries that are imported
* [Pandas Documentation](https://pandas.pydata.org/docs/)
* [Numpy Documentation](https://numpy.org/doc/)
* [Folium Documentation](https://python-visualization.github.io/folium/)
* [Json Documentation](https://docs.python.org/3/library/json.html)
* [Statsmodels Documentation](https://www.statsmodels.org/stable/)
* [Scipy Documentation](https://scipy.github.io/devdocs/index.html)
* [Matplotlib Documentation](http://matplotlib.org/)
* [Seaborn Documentation](http://seaborn.pydata.org/)

In [ ]:
#Importing needed libraries
import pandas as pd                                                            # used to provide major datastructure pd.DataFrame() to store the datasets
import numpy as np                                                             # used for numerical calculations and fast array manipulations
import folium                                                                  # used for spatial data visualizations
import json                                                                    # used for loading json data correctly
import statsmodels.api as sm                                                   # used to run multivariate linear regression
from scipy.stats import pearsonr, spearmanr                                    # used to run 'pearson' and 'spearman' association tests of numerical variables on two variables
from statsmodels.stats.multitest import multipletests                          # used to run multiple tests of p-values for multiple variables
import matplotlib.pyplot as plt                                                # used for plotting and visualizing our data
import matplotlib.dates as mdates                                              # used to create right ticks for x-axes in diagrams
import seaborn as sns                                                          # used for plotting and visualizing our data
from IPython.display import Markdown, display                                  # used to print stuff with markdown/HTML formatting for bold text and colored text
def printmd(string):
    display(Markdown(string))

### Loading the raw data

In [ ]:
#Importing the raw corona data from Germany
corona_df = pd.read_csv('../data/raw/corona/de_corona.csv', sep = '\t')
corona_df.name = 'corona_df'

#Importing the raw weather data for the countries, Germany, Netherlands, Sweden and Denmark
weather_df = pd.read_csv("../data/raw/weather/weather.csv")
weather_df.name = 'weather_df'

#Loading in the metadata json using the Python json library
with open('../data/raw/metadata/de_metadata.json','r', encoding="utf8") as f:
    country_metadata=json.load(f)

#Creating a folium map (called de_map) that is based around Germany and uneditable in terms of placement and zoom
de_map = folium.Map(location = [51.1657, 10.4515], zoom_start = 6, crs = 'EPSG3857', 
    zoom_control = False, scrollWheelZoom = False, dragging = False)

#Loading in the geojson that contains data for the regions and borders of Germany and adding it to the folium map
folium.GeoJson('../data/raw/shapefiles/de.geojson', name = "geojson").add_to(de_map)
folium.LayerControl().add_to(de_map);

### Task 0: Data filtering and cleaning

The data analysis done in this notebook is done with a handful of different datasets:

> CSV: Corona (DE) - Contains the Number of new infections (per day) and Number of new casualties (per day) filtered by day and region in Germany for each day in the period `2020-01-02` to `2021-02-21`.
>
> CSV: Weather - Contains information about several indicators of weather conditions for each region in Germany, Denmark, Sweden and Netherlands for each day in the period `2020-02-13` to `2021-02-21`
>
> JSON: Metadata (DE) - Contains more information on the population in the different regions in Germany
>
> GEOJSON: Geojson (DE) - Holds the geojson data for the different regions in Germany

### Initial inspection of the datasets

In [ ]:
row, cols = weather_df.shape
print("Number of Rows: " + str(row))
print("Number of Columns: " + str(cols))
weather_df.head()

We can see that the `weather` dataset contains `9` different variables for the countries Germany, Denmark Sweden and Netherlands spread across `20220` rows and `9` columns. Each row contains `7` different weather measurements for a particular day in a specific country and region:
> `Date` (YYYY-DD-MM): Is the day of which the weather measurements were made and reported
>
> `Iso3166-2`: Is the [ISO 3166-2](https://en.wikipedia.org/wiki/ISO_3166-2) code for the region for which the weather measurements were made
>
> `Relative Humidity Surface` (%): Is the **daily average** relative humidity of the surface
>
> `Solar Radiation`(W/m^2): Is the **daily sum** of the solar radiation from the sun
>
> `Surface Pressure` (Pa): Is the **daily sum** of the atmospheric pressure at the surface of the earth
>
> `TemperatureAboveGround` (º K): Is the **daily average** of the temperature above ground
>
> `Total Precipitation` (mm): Is the **daily sum** of the total precipitation 
>
> `UV Index` (Numerical value): Is the **daily sum** of the strength of the ultraviolet radiation
>
> `Wind Speed`(m/s): Is the **daily average** of the wind speed

In [ ]:
row, cols = corona_df.shape
print("Number of Rows: " + str(row))
print("Number of Columns: " + str(cols))
corona_df.head()

We can see that the `corona` dataset contains `4` different variables for corona reports for Germany spread across `5602` rows and `4` columns. Each row contains `2` different corona reports for a particular day in a specific country and region:
> `Date` (YYYY-DD-MM): Is the day of which the corona report was made
>
> `Region Code`: Is the region code for the region for which the corona report was made
>
> `Confirmed Addition`: Is the number of new confirmed infections of corona for the specific day in the specified region
>
> `Deceased Addition`: Is the number of newly confirmed deaths for the specific day in the specified region

### Sanity check
Checking to see if there are any missing (Na or NaN) values in either of the two datasets \
\
We also check if there are any duplicate rows in either of the two datasets. \
\
Next we also check if any of the reported numerical values are negative. It would not make sense to e.g., have a negative UV Index or a negative comfirmed COVID-19 cases.

### Checking for NaN values:

In [ ]:
datasets = [weather_df, corona_df]
for dataset in datasets:
    if dataset.name == "weather_df":
        printmd("### **Weather Dataset**")
    else: 
        printmd("### **Corona Dataset**")

    print(dataset.isnull().any())
    print("-----------------------------------------------")
    printmd( "##### <span style='color:red'> ❌ **Oh No! There are missing data values in the dataset!**</span>" if dataset.isnull().any().any() 
            else
            "##### <span style='color:lightgreen'> ✅ **Great! There are no missing data values in the dataset!**</span>")
    printmd("____")

### Checking for duplicates:

In [ ]:
for dataset in datasets:
    if dataset.name == "weather_df":
        printmd("### **Weather Dataset**")
    else: 
        printmd("### **Corona Dataset**")

    print(dataset.duplicated())
    print("-----------------------------------------------")
    printmd( "##### <span style='color:red'> ❌ **Oh No! There are duplicate data values in the dataset!**</span>" if dataset.duplicated().any()
            else
            "##### <span style='color:lightgreen'> ✅ **Great! There are no duplicate data values in the dataset!**</span>")
    printmd("____")

### Checking for values negative reports

In [ ]:
for dataset in datasets:
    if dataset.name == "weather_df":
        printmd("### **Weather Dataset**")
    else: 
        printmd("### **Corona Dataset**")
    dataset = dataset.select_dtypes(include=np.number)

    print((dataset < 0).any())
    print("-----------------------------------------------")
    printmd( "##### <span style='color:red'> ❌ **Oh No! There are data values that shouldn't be negative in the dataset!**</span>" if (dataset < 0).any().any() 
            else
            "##### <span style='color:lightgreen'> ✅ **Great! There are no data values that shouldn't be negative in the dataset!**</span>")
    printmd("____")

### Fixing variable names

We noticed that the variables in the weather dataset had a different naming scheme than the corona dataset so in order to make it more seamless we rename the variables to all have the same naming scheme.

In [ ]:
weather_df.rename(columns={
    'date': 'date', 
    'iso3166-2': 'region_code',
    'RelativeHumiditySurface': 'relative_humidity_surface', 
    'SolarRadiation': 'solar_radiation', 
    'Surfacepressure': 'surface_pressure', 
    'TemperatureAboveGround': 'temperature_above_ground', 
    'Totalprecipitation': 'total_precipitation', 
    'UVIndex': 'UV_index', 
    'WindSpeed': 'wind_speed'}, inplace=True)

Since there are no missing or falsely reported data values we can move on to cleaning and filtering the data.

#### _Let's start by cleaning and filtering the corona data for Germany:_

We first start of by creating to dictionaries that are going to help us to map both regions and their populations to the dataframes:

In [ ]:
#Creating a dictionary that contains the full names of the different regions as keys and their respective iso3166-2 code as values
region_map = {country_metadata["country_metadata"][i]["covid_region_code"]: 
    country_metadata["country_metadata"][i]["iso3166-2_code"] for i in range(len(country_metadata["country_metadata"]))}

#Creating a dictionary that contains the full names of the different regions as keys and their respective populations as values
population_map = {country_metadata["country_metadata"][i]["iso3166-2_code"]: 
    country_metadata["country_metadata"][i]["population"] for i in range(len(country_metadata["country_metadata"]))}

Now with the region_map we can create extra columns to add the iso3166-2 code to the corona dataframe as it only contained to full region codes. \
\
We can also use the population_map to add a column with the population of each of the regions to the corona dataframe. \
\
With the population column we can create a column that contains the cases and deaths per capita using the confirmed addition and the population column we just created.

In [ ]:
#Using the region_map dictionary to create a new column with the respective iso3166-2 code for each region based on the full region name
#from the region_code column
corona_df["region_code"] = corona_df["region_code"].map(region_map)

#Using the population_map dictionary to create a new column with the respective population for each region based on the iso3166-2 code
#from the iso3166-2 column
corona_df["population"] = corona_df["region_code"].map(population_map)

#Also adding a cases and deaths per capita column that is created using the confirmed covid cases and deaths respectively divided by the population in that region
corona_df["cases_pc"] = corona_df["confirmed_addition"] / corona_df["population"]
corona_df["deaths_pc"] = corona_df["deceased_addition"] / corona_df["population"]

#### _Now moving on to the weather dataset:_

We start off by first filtering out all the other countries from the dataset as we are not interested in their data.\
\
We can then convert the temperature measurements from º K to º C, we do that by subtracting `273.15` from each row value in the temperature measurements

<span style='color:IndianRed'>Please note! Be careful to not run the following cell more than once as this will keep doing the subtractions which could lead to negative and or NaN temperature values</span>

In [ ]:
#Filtering out all the weather data that is not relevant as we are only interested in weatherdata from Germany
weather_df = weather_df[weather_df["region_code"].str.startswith("DE")]

#Converting the temperature from Kelvin to Celsius
weather_df["temperature_above_ground"] = weather_df["temperature_above_ground"] - 273.15

Finally we can merge the corona_df and the weather_df to create on big dataframe/dataset that contains all the data that we need.

In [ ]:
#Merging the weatherdata with the coronadata to create one dataframe with all the data that we need
merged_df = corona_df.merge(weather_df, on = ["date", "region_code"])
merged_df

Now that we have merged the two datasets we want to check how many rows we lost from each of the two datasets. \
\
We do that by subtracting the number of rows from each of the two datasets by the number of rows in the merged dataset.

In [ ]:
weather_lost = weather_df.shape[0]-merged_df.shape[0]
corona_lost = corona_df.shape[0]-merged_df.shape[0]
print(f"We lost {weather_lost} rows from the weather dataset.")
print(f"We lost {corona_lost} rows from the corona datset")
print(f"We lost {weather_lost+corona_lost} rows in total")

As we can see we lost 442 rows from the two datasets by merging.
This could be due to the fact that the corona dataset starts from the 2nd of January \
whereas the weather dataset starts on the 13th of February and therefore some of the corona data is lost. \
It could also be possible that on the days that do match one of the datasets does not contain data on the same region as the other dataset, resulting in more dropped rows.


### Outputting and re-loading filtered and cleaned data

Now having cleaned and filtered to two datasets we can create new `.csv` files that we can use for the rest of this notebook. \
\
We do this by using the built-in pandas function `pd.to_csv` that will automatically convert the pandas dataframe to a `.csv` file \
\
We will also create shortcuts to those files so the code becomes more human readable.

In [ ]:
# The pd.to_csv function is run by first specifying the file path and name that it should create.
# The index = False argument specifies to the function that the .csv file should not include the index that the pandas dataframe has
corona_df.to_csv("../Data/processed/de_corona.csv", index = False)
weather_df.to_csv("../Data/processed/de_weather.csv", index = False)
merged_df.to_csv("../Data/processed/de_corona_weather.csv", index = False)

# Now creating the shortcuts to the processed datasets
corona_path = "../Data/processed/de_corona.csv"
weather_path = "../Data/processed/de_weather.csv"
merged_path = "../Data/processed/de_corona_weather.csv"

We can now read in and overwrite the name variables of the datasets to make sure that the data we use has been filtered and cleaned correctly:

In [ ]:
corona_df = pd.read_csv(corona_path)
weather_df = pd.read_csv(weather_path)
merged_df = pd.read_csv(merged_path)

### Task 1: Single variable analysis

#### Creating dataframes for plotting

##### Creating dataframes with different time frequency

In [ ]:
#changing values in date column to be datetime type
merged_df["date"] = pd.to_datetime(merged_df["date"])

#creating dataframes with dates and summed confirmed addition
#all the dates
merged_dfbydate=merged_df.groupby(by='date')['confirmed_addition'].sum().reset_index()
#one date per week
merged_dfbyweek=merged_dfbydate.groupby(pd.Grouper(key='date',freq='W')).sum().reset_index()
#one date per month
merged_dfbymonth=merged_dfbydate.groupby(pd.Grouper(key='date',freq='M')).sum().reset_index()


##### Creating dataframes with weather data for different time frequency

In [ ]:
#now we create dataframes with weather data but only for some dates we will calculate the mean of the variable for that date
#daily
temp_dfbydate=merged_df.groupby(by='date').mean()
temp_dfbydate=temp_dfbydate.loc[:,temp_dfbydate.columns!='confirmed_addition']
merged_dfbydate=merged_dfbydate.merge(temp_dfbydate,on='date')
#weekly
temp_dfbyweek=merged_df.groupby(by='date').mean()
temp_dfbyweek=temp_dfbyweek.loc[:,temp_dfbyweek.columns!='confirmed_addition']
merged_dfbyweek=merged_dfbyweek.merge(temp_dfbyweek,on='date')
#monthly
temp_dfbymonth=merged_df.groupby(by='date').mean()
temp_dfbymonth=temp_dfbymonth.loc[:,temp_dfbymonth.columns!='confirmed_addition']
merged_dfbymonth=merged_dfbymonth.merge(temp_dfbymonth,on='date')

##### Creating dataframes with COVID-19 data for different time frequency

In [ ]:
#creating dataframes with only date, region_code and confirmed addition columns grouped by date and region_code
#daily
cor_dfbydate=merged_df.groupby(['date','region_code']).mean()
cor_dfbydate.reset_index(inplace=True)
cor_dfbydate=cor_dfbydate.loc[:,['date','region_code','confirmed_addition']]
cormerged_dfbydate=cor_dfbydate.merge(merged_dfbydate.loc[:,'date'],on='date')
#weekly
cor_dfbyweek=merged_df.groupby(['date','region_code']).mean()
cor_dfbyweek.reset_index(inplace=True)
cor_dfbyweek=cor_dfbyweek.loc[:,['date','region_code','confirmed_addition']]
cormerged_dfbyweek=cor_dfbyweek.merge(merged_dfbyweek.loc[:,'date'],on='date')
#monthly
cor_dfbymonth=merged_df.groupby(['date','region_code']).mean()
cor_dfbymonth.reset_index(inplace=True)
cor_dfbymonth=cor_dfbymonth.loc[:,['date','region_code','confirmed_addition']]
cormerged_dfbymonth=cor_dfbymonth.merge(merged_dfbymonth.loc[:,'date'],on='date')

In [ ]:
#setting the index to be the date column
merged_df.set_index('date', inplace=True)
cormerged_dfbyweek.set_index('date', inplace=True)
cormerged_dfbymonth.set_index('date', inplace=True)
cormerged_dfbydate.set_index('date', inplace=True)

#### Plotting corona data
After plotting corona data grouped by region. The plot was not readable and we decided that it will be better if we split the plot into 2 plots with 8 regions. We divided them according to the population. 

In [ ]:
#Plotting corona data for all regions daily
cormerged_dfbydate.groupby('region_code')['confirmed_addition'].plot(x=cormerged_dfbydate.index,kind="line",legend=True, figsize=(15,13),title='Corona data daily',ylabel='Confirmed addition')
plt.show()
#Plotting corona data for all regions weekly
cormerged_dfbyweek.groupby('region_code')['confirmed_addition'].plot(x=cormerged_dfbyweek.index,kind="line",legend=True, figsize=(15,13),title='Corona data weekly',ylabel='Confirmed addition')
plt.show()
#Plotting corona data for all regions monthly
cormerged_dfbymonth.groupby('region_code')['confirmed_addition'].plot(x=cormerged_dfbymonth.index,kind="line",legend=True, figsize=(15,13),title='Corona data monthly',ylabel='Confirmed addition')
plt.show()

#Plotting corona data for 8 more populated German regions weekly
newmerged_df=cormerged_dfbyweek.loc[(cormerged_dfbyweek['region_code']=='DE-SN')|(cormerged_dfbyweek['region_code']=='DE-RP')|(cormerged_dfbyweek['region_code']=='DE-NW')|(cormerged_dfbyweek['region_code']=='DE-NI')|(cormerged_dfbyweek['region_code']=='DE-HE')|(cormerged_dfbyweek['region_code']=='DE-BY')|(cormerged_dfbyweek['region_code']=='DE-BW')|(cormerged_dfbyweek['region_code']=='DE-BE')]
newmerged_df.groupby('region_code')['confirmed_addition'].plot(x=newmerged_df.index,kind="line",legend=True, figsize=(15,13),title='Corona data in the 8 more populated German regions weekly',ylabel='Confirmed addition')
plt.show()
#Plotting corona data for 8 less populated German regions weekly
newmerged_df=cormerged_dfbyweek.loc[(cormerged_dfbyweek['region_code']=='DE-BB')|(cormerged_dfbyweek['region_code']=='DE-HB')|(cormerged_dfbyweek['region_code']=='DE-HH')|(cormerged_dfbyweek['region_code']=='DE-MV')|(cormerged_dfbyweek['region_code']=='DE-SL')|(cormerged_dfbyweek['region_code']=='DE-ST')|(cormerged_dfbyweek['region_code']=='DE-SH')|(cormerged_dfbyweek['region_code']=='DE-TH')]
newmerged_df.groupby('region_code')['confirmed_addition'].plot(x=newmerged_df.index,kind="line",legend=True, figsize=(15,13),title='Corona data in the 8 less populated German regions weekly',ylabel='Confirmed addition')
plt.show()

#### Plotting weather data
We already created dataframes with weather data grouped by date with average values for all the weater variables.
(RelativeHumiditySurface,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed)


In [ ]:
#we are plotting one subplot for each variable from the list:
weather=[['relative_humidity_surface','surface_pressure','temperature_above_ground'],['total_precipitation','UV_index','wind_speed']]
#We create figure for every time frequency  and then we plot 'date' column on X axis and weather data on Y axis
#plotting weather variables daily
fig, ax = plt.subplots(2,3,figsize=(23,13))
fig.suptitle(' Weather data daily ', fontsize=20)
for i in range(2):
    for j in range(3):
        ax[i][j].plot(merged_dfbydate['date'],merged_dfbydate[weather[i][j]])
        ax[i][j].set_title(weather[i][j])
        ax[i][j].set_xlabel('date')
#plotting weather variables weekly
fig1, ax1 = plt.subplots(2,3,figsize=(23,13))
fig1.suptitle(' Weather data weekly ', fontsize=20)
for i in range(2):
    for j in range(3):
        ax1[i][j].plot(merged_dfbyweek['date'],merged_dfbyweek[weather[i][j]])
        ax1[i][j].set_title(weather[i][j])
        ax1[i][j].set_xlabel('date')
#plotting weather variables monthly
fig2, ax2 = plt.subplots(2,3,figsize=(23,13),num='Weather data monthly')
fig2.suptitle(' Weather data monthly ', fontsize=20)
for i in range(2):
    for j in range(3):
        ax2[i][j].plot(merged_dfbymonth['date'],merged_dfbymonth[weather[i][j]])
        ax2[i][j].set_title(weather[i][j])
        ax2[i][j].set_xlabel('date')
     


For `RelativeHumiditySurface` we can see that there is an increase in the values throughout the year, they reach the highest values in the colder months, and then decrease drastically during March. This decrease is more difficult to notice in the monthly weather data plot, compared to the daily plot where the difference is showcased more clearly.

While observing `Surfacepressure` we can mostly see similar values during the whole year with a few occasional falls. Again, if we look at the monthly weather data plot we can notice that the values are not as equally distributed since those falls change the mean of the pressure for that month.

The graph for `TemperatureAboveGround` shows what typical temperatures for each season are, during the warmer months our values keep increasing, reaching their peak in August and then decreasing during the colder months to reach the lowest values in February.

`Totalpercipitation` plot was the most challenging one to observe since the daily, weekly, and monthly weather data plots look dissimilar. If we look at the daily weather data plot, we can see that the values fluctuate drastically through all of the months reaching both low and high values. For this reason, we can see bigger differences in weekly and monthly means which would then explain the overall difference in graphs.

`UVIndex` values can be compared to the `TemperatureAboveGround` values in the way that both of them are higher during warm months and lower during cold ones. The difference would be that `UVIndex` has a more steep line and therefore declines more rapidly to the low values, for example, after the peak that it reaches in August.

Finally, `WindSpeed` fluctuates daily so we have numerous jumps that result in different means for weekly and monthly values. At the same time, we can notice that during winter we have overall greater values compared to the other seasons.

In [ ]:
#Pulling data from our datafile from OWID and merging it to our data frame.
owid = pd.read_csv("../Data/external/de_our_world_in_data_covid.csv")
owid = owid[owid["iso_code"] == "DEU"][['date', 'stringency_index']].dropna()
df = pd.read_csv("../Data/processed/de_corona_weather.csv")
df = df.merge(owid, on="date")
df.set_index('date', inplace=True)


#Visualizing the stringency index.
fig, ax = plt.subplots(figsize=(16, 8))
ax.plot(df.index, df["stringency_index"])
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=(1, 1)))
plt.xticks(rotation = 45)
fig.suptitle("Stringency Index for DE by month", fontweight = "bold");

### Task 2: Associations

While researching whether there is a significant statistical association in Germany between weather data and infection rates, the most important variables that we took in consideration were:

> `R-squared` (%): analyzes how much the changes in the independent variable can explain the changes in the dependent variable
>
> `Adj. R-squared` (%): is R-squared, but modified for the number of variables
>
> `coef`: measures how change in one variable affect the independent variable; if negative, our variables will have an inverse relationship, meaning that if one increases the other decreases
>
> `t`: measurement of the precision with which the coefficient is measured
>
> `p > |t|`: p value, based on the t statistic, shows how likely is our coefficient measured through our model

In [ ]:
# Variables from the collected weather data
Xs = ['relative_humidity_surface', 'solar_radiation', 'surface_pressure', 'temperature_above_ground', 
    'total_precipitation', 'UV_index', 'wind_speed']

# Adding a constant term to both our dataframe and the Xs list
merged_df = sm.add_constant(merged_df)
Xs.append('const')

# Performing the ordinary least squares regession to fit the regression line to our data
est = sm.OLS(merged_df['confirmed_addition'], merged_df[Xs], hasconst=True).fit()
est.summary()

### _Plotting weather variables with infection rates:_

In [ ]:
#Visualizing the UV index in comparison with confirmed cases.

#Creating the plot
fig, ax = plt.subplots(figsize=(16, 8))

#Adding the first variable, UV Index
ax.plot(df.index, df["UV_index"], "#fec44f", label="UV Index")

#Adding a second variable and adding its own y-axis to show the confirmed additions
ax2 = ax.twinx()
ax2.plot(df.index, df["confirmed_addition"], color ="#2c7fb8", label="Confirmed Addition")
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=(1, 1)))

#Adding the main title to the plot as well as the x and y labels
fig.suptitle("UV Index for DE and number of confirmed cases", fontweight = "bold")
ax.set_ylabel("UV Index", fontweight = "bold")
ax2.set_ylabel("Confirmed addition", fontweight = "bold")
ax.set_xlabel("Date", fontweight = "bold")

#Adding the legend and specifying the position
fig.legend(loc = "upper left", borderaxespad = 8);

In [ ]:
#Visualizing the UV index with the amount of confirmed cases

#Creating a separate dataframe to use when plotting that only contains the UV index and confirmed addition
rel_df = merged_df.groupby(by = "UV_index" ) ["confirmed_addition"].mean().round().reset_index()

#Rounding the UV Index to make it easier to plot as the UV Index values had a lot decimal values
rel_df["UV_index"] = rel_df["UV_index"].round()

#Log transforming the confirmed addition
rel_df["confirmed_addition"] = np.log(rel_df["confirmed_addition"]+ 1)

#Finally plotting the two values using the seaborn relationship plot function as a lineplot
sns.relplot(data=rel_df, x = "UV_index", y = "confirmed_addition", kind= "line");

#### _Association report:_

Based on the `0.263` value of `R-squared`, weather variables can be used to explain `26.3%` of cases. `Adj. R-squared` with the value of `0.262` is almost the same as `R-squared` since we are dealing with a small number of variables and thus `Ajd.  R-squared` can be ruled out as not relevant.

We have a `significance threshold` of `0.05` which is used to determine whether a variable has a significant association with our dependent value, `confirmed_addition`. If the variables have a `P-value` less than this (`0.05`) we cannot deny that there could be a correlation. Out of `8` independent variables, `temperature_above_ground` is the only one that can be excluded, since its `P-value` is `0.133`. 

We have chosen to focus on `relative_humidity_surface` and `UV_index` since they have high spikes in both negative and positive correlations with the `confirmed_additions`. The coefficients level for `relative_humidity_surface` shows that when it increases there is also a rise in the number of COVID-cases, whereas for spikes of `UV_index` values, there is a related decrease in COVID-cases.

### Task 3: Map Visualizations

In order to make it easier to create multiple maps we define a function. \
In this function you can specify what condition you want to visualize along with what color scale to use

In [ ]:
#Creating a function for creating Folium maps with choropleths to make it easier 
# and more compact to create multiple maps for different variables.
def choropleth(data, condition, color):
    geo_json_path='../data/raw/shapefiles/de.geojson'
    f = folium.Figure(width=1000, height=600)
    m = folium.Map(location=[51.1657, 10.4515],zoom_start=6,crs='EPSG3857',
    zoom_control=False, scrollWheelZoom=False,dragging=False,tiles="cartodbpositron").add_to(f)

    folium.Choropleth(
        geo_data = geo_json_path,
        name = condition,
        data = data,
        columns = ["region_code", condition],
        key_on = "properties.iso_3166_2",
        fill_color = color,
        fill_opacity = 0.7,
        line_opacity = 0.2,
        legend_name = f"{condition.upper().replace('_', ' ')}",
    ).add_to(m)
    return m

#### _Making a map of the UV index across the different regions_

In [ ]:
choropleth(merged_df, "UV_index", "PuBu")

#### _Making a map of the amount of COVID cases per region_

In [ ]:
choropleth(merged_df, "cases_pc", "OrRd")

### Task 4: Open Question

We want to compare Germany to Sweden as Sweden is the country that is the furthest away from Germany and also has a significantly different climate and geographical features \
So we just do the same things to manipulate the German data to the Sweden data, so we will not go into as much detail with comments as we did with Germany.

In [ ]:
se_weather_df = pd.read_csv("../Data/raw/weather/weather.csv")
#Loading the sweden data as a pandas dataframe

se_weather_df.rename(columns={
    'date': 'date', 
    'iso3166-2': 'region_code',
    'RelativeHumiditySurface': 'relative_humidity_surface', 
    'SolarRadiation': 'solar_radiation', 
    'Surfacepressure': 'surface_pressure', 
    'TemperatureAboveGround': 'temperature_above_ground', 
    'Totalprecipitation': 'total_precipitation', 
    'UVIndex': 'UV_index', 
    'WindSpeed': 'wind_speed'}, inplace=True)

#Making a dataset that only contains weatherdata from Sweden
se_weather_df = se_weather_df[se_weather_df['region_code'].str.startswith('SE')]



In [ ]:
se_corona_df=pd.read_csv('../data/raw/corona/se_corona.csv',sep='\t')
with open('../data/raw/metadata/se_metadata.json','r', encoding="utf8") as f:
    country_metadata=json.load(f)

region_map = {country_metadata["country_metadata"][i]["covid_region_code"]: 
    country_metadata["country_metadata"][i]["iso3166-2_code"] for i in range(len(country_metadata["country_metadata"]))}
se_corona_df["region_code"] = se_corona_df["region_code"].map(region_map)

In [ ]:
se_merged_df = se_corona_df.merge(se_weather_df)
#Reversing the dates so they start from 2020 and not 2021
se_merged_df = se_merged_df.iloc[::-1]
se_merged_df.set_index("date", inplace = True)

In [ ]:
#Visualizing the UV index in comparison with confirmed cases.


#Creating the plot
fig, ax = plt.subplots(figsize=(16, 8))

#Adding the first variable, UV Index
ax.plot(se_merged_df.index, se_merged_df["UV_index"], "#fec44f", label="UV Index")

#Adding a second variable and adding its own y-axis to show the confirmed additions
ax2 = ax.twinx()
ax2.plot(se_merged_df.index, se_merged_df["confirmed_addition"], color ="#2c7fb8", label="Confirmed Addition")
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=(1, 1)))

#Adding the main title to the plot as well as the x and y labels
fig.suptitle("UV Index for SE and number of confirmed cases", fontweight = "bold")
ax.set_ylabel("UV Index", fontweight = "bold")
ax2.set_ylabel("Confirmed addition", fontweight = "bold")
ax.set_xlabel("Date", fontweight = "bold")

#Adding the legend and specifying the position
fig.legend(loc = "upper left", borderaxespad = 8);

As we can see the UV index in Sweden also has significant impact on the number of confirmed COVID-19 cases. \
It shows that there is a strong correlation between these two variables and that it is not affected by the geographical location of the country.